## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urlib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음

In [7]:
from urllib.request import urlopen, Request

#도시별 날씨 검색 함수
def get_weather(city):
    #기상청 홈페이지 도시별 날씨
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text =text[text.find(f'>{city}</a>'):]

    #기온 가져오기 반복문 13번
    for i in range(7):
        text = text[text.find('<td>')+1:]

    #찾은 지점부터 3번째 값
    start = 3
    #사이값 찾은 뒤에 있는 </tb> 값
    end = text.find('</td>') #시작지점과 종료지점 사이값
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다')

    #print(text[start:end])

#습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다')



get_weather('부산')

부산의 현재 기온은 11.4˚C 입니다
부산의 현재 습도는 23% 입니다


### OpenAPI 크롤링

- 공공데이터포탈
    - https://www.data.go.kr/
    - 부산광역시 버스데이터현황

```python
reponse = request.get(total_url,verify=False)
```

- 문제점
    - https를 일반 request로 부르면 ssl 오류 발생
    - 해결하려면 외부모듈 request를 사용, verify = False 옵션을 지정

In [8]:
# from urllib.request import urlopen, Request # 안써도 무방한데 조금 더 안전하다.
import requests
from urllib.parse import quote, unquote, urlencode # 한글을 urlencode  변환하는 함수 롯데 --> %EB%A1%AF%EB%8D%B0
import json
import ssl

# def getRequestUrl(url):
#     req = Request(url)
    
#     try:
#         res = urlopen(req)
#         if res.getcode() == 200:
#             return res.read().decode('utf-8') #읽어온 데이터를 리턴
#     except Exception as e:
#         print(e)
#         return None #돌아온 데이터가 없으면 실행 안함

#stationName 정류소 이름
def getDataPortalSearch(stationName,type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" +urlencode(
        {
            'serviceKey' : 'HVto27eamBEo1E7tjkDl7GVSuHLlwZcM2XpUQB691yS14zX9Wu1OK5ZNqnOzK5REs8adTJvOTuUNABJgDw8x7Q==',
            'pageNo' : '1',
            'numOfRows' : '10',
            'resultType': type,
            'stationLoc' : stationName

        }
    )

    total_url = api_url + queryString
    reponse = requests.get(total_url, verify=False) # verify 검증
    return reponse.text

    # api_url += '?serviceKey=HVto27eamBEo1E7tjkDl7GVSuHLlwZcM2XpUQB691yS14zX9Wu1OK5ZNqnOzK5REs8adTJvOTuUNABJgDw8x7Q%3D%3D'
    # api_url += '&pageNo=1'
    # api_url += '&numOfRows=10'
    # api_url += f'&resultType={type}'
    # api_url += f'&stationLoc=%EB%B0%B1%ED%99%94%EC%A0%90={stationName}'

    # response = getRequestUrl(api_url)

    # if response == None:
    #     return None
    # else:
    #     return json.loads(response)
    # return api_url

# print(getDataPortalSearch('롯데백화점','json'))

result = getDataPortalSearch('롯데백화점','json')

json_data = json.loads(result)
#딕셔너리 형태로 데이터 출력
station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item'] # [] 안에 있는 데이터를 받아올 수 있다. 
# quote ('롯데')
# try:
#     result = getDataPortalSearch('롯데백화점','json')
#     json_data = json.loads(result)
# except Exception as e:
#     print(e)

for item in station_data:
    print(item)

{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}


c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>' + item['stationLoc'] +'</h4>' + item['stationNum'] +'<br>' + item['addr']
```

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 ㄹ'{}' 호환 안됨!
    - 구식방법인 물자열 결합방식으로 해결

In [15]:
import folium

if len(station_data) > 0: # 만약 station_data 길이가 0보다 크면 
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']


m  = folium.Map(location=[center_lat, center_lng], zoom_start=12)

for item in station_data:
    stop_str = '<h4>' + item['stationLoc'] +'</h4>' + item['stationNum'] +'<br>' + item['addr']
    iframe = folium.IFrame(stop_str)
    popip = folium.Popup(iframe, min_width = 200, max_width = 200) #팝업 사이즈 지정
    # stop_str += item['stationNum'] +'<br>'
    # stop_str += item['addr']
    folium.Marker(location=[item['lat'],item['lng']], popup=item['stationLoc'],
    icon=folium.Icon(color='red', icon='pushpin')).add_to(m)
m

### Beautifulsoup (version4)

웹크롤링을 편하게 해주는 도구 (라이브러리)

pip install beautifulsoup4

In [10]:
# !pip install beautifulsoup4

#### 모듈 import

```python
from bs4 Beautifulsoup
```

In [11]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9'

response = requests.get(url)

if response.status_code == 200: #webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') # parser 어떤 값을 분석해서 보기 좋게 만들어줌
    #print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())
else:
    print(f'Error:{response.status_code}')


제임스웹 우주망원경 질문이요


In [12]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9'

response = requests.get(url)

if response.status_code == 200: #webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser') # parser 어떤 값을 분석해서 보기 좋게 만들어줌
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error:{response.status_code}')


제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
빅뱅과 제임스웹 관측 질문
제임스 웹 망원경의 종류 알려주세요
제임스웹에 대한 질문입니다
제임스 웹 우주망원경
